In [1]:
from urllib.parse import quote
from urllib import request
import json
import xlwt

In [2]:
# 将返回的poi数据装入集合返回
def hand(poilist, result, adcode, counter):
    pois = result['pois']
    
    for i in range(len(pois)):
        #print(pois[i]['adcode'])
        ad = pois[i]['adcode']
        if ad == adcode:
            #print('in')
            poilist.append(pois[i])
            counter = counter + 1
            
    return counter

In [3]:
# 单页获取pois
def getpoi_page(minlng, maxlat, maxlng, minlat, types, page):
    req_url = poi_search_url + 'key=' + amap_web_key + '&extensions=all&keywords=' + quote(
        types) + '&polygon=' + str(minlng) + ',' + str(maxlat) + '|' + str(maxlng) + ',' + str(minlat) + '&offset=20' + '&page=' + str(
        page) + '&output=json'

    data = ''
    #print(quote(types))
    with request.urlopen(req_url) as f:
        data = f.read()
        data = data.decode('utf-8')
    #print(req_url)
    return data

In [4]:
# 根据grid和分类关键字获取poi数据
def getpois(minlng, maxlat, maxlng, minlat, keywords, adcode, counter):
    #counter = 0
    i = 1
    poilist = []
    
    while True:  # 使用while循环不断分页获取数据
        result = getpoi_page(minlng, maxlat, maxlng, minlat, keywords, i)
        #print(result)
        result = json.loads(result)  # 将字符串转换为json
        #print(result['count'])  
        
        #if result['count'] == 900:
            #generate_grid()
        
        if result['count'] == '0':
            break
        counter = hand(poilist, result, adcode, counter)
            
        i = i + 1
    print('counter='+str(counter))
    return poilist, counter

In [5]:
# 数据写入excel
def write_to_excel(poilist, classfield, num, adcode, time):
    # 一个Workbook对象，这就相当于创建了一个Excel文件
    book = xlwt.Workbook(encoding='utf-8', style_compression=0)
    sheet = book.add_sheet(classfield, cell_overwrite_ok=True)

    sheet.write(0, 0, '用地类型（大类）')
    sheet.write(0, 1, '用地类型（中类）')
    sheet.write(0, 2, '用地类型（小类）')
    sheet.write(0, 3, 'x')
    sheet.write(0, 4, 'y')
    sheet.write(0, 5, 'count')
    sheet.write(0, 6, 'name')
    sheet.write(0, 7, '类型')
    sheet.write(0, 8, '省')
    sheet.write(0, 9, '市')
    sheet.write(0, 10, '区')
    #sheet.write(0, 11, '地址')


    for i in range(len(poilist)):
        types = poilist[i]['type']
        big = str(types).split(";")[0]
        median = str(types).split(";")[1]
        small = str(types).split(";")[2]
        location = poilist[i]['location']
        name = poilist[i]['name']
        lng = str(location).split(",")[0]
        lat = str(location).split(",")[1]
        leixing = poilist[i]['type']
        pname = poilist[i]['pname']
        city = poilist[i]['cityname']
        qu = poilist[i]['adname']
        #postcode = poilist[i]['address']


        # 每一行写入
        sheet.write(i + 1, 0, big)      #用地类型（大类）
        sheet.write(i + 1, 1, median)     #用地类型（中类）
        sheet.write(i + 1, 2, small)     #用地类型 小类
        sheet.write(i + 1, 3, lng)      #x
        sheet.write(i + 1, 4, lat)      #y
        sheet.write(i + 1, 5, 1)
        sheet.write(i + 1, 6, name)     #name
        sheet.write(i + 1, 7, leixing)  #类型
        sheet.write(i + 1, 8, pname)       #省
        sheet.write(i + 1, 9, city)  # 市
        sheet.write(i + 1, 10, qu)  # 区
        #sheet.write(i + 1, 11, postcode)  #地址


    # 最后，将以上操作保存到指定的Excel文件中
    book.save(r'' + classfield + '_' + str(adcode) + '_' + str(num) + '格_' + str(time) + '次' + '.xls')
    return book

In [6]:
import numpy as np
import pandas as pd
import xlwt

In [7]:
#如果方格数据大于900，进行二次划分
def generate_grids(start_long,start_lat,end_long,end_lat, num, adcode, time):
    
    assert start_long < end_long,'需要从左上到右下设置经度，start的经度应小于end的经度'
    assert start_lat > end_lat,'需要从左上到右下设置纬度，start的纬度应大于end的纬度'
    #assert resolution>0,'resolution应大于0'
    
    centerlng = round(float((end_long+start_long)/2),6)
    centerlat = round(float((end_lat+start_lat)/2),6)
    minlng = round(float(start_long),6)
    maxlng = round(float(end_long),6)
    minlat = round(float(end_lat),6)
    maxlat = round(float(start_lat),6)
    
    book = xlwt.Workbook(encoding='utf-8', style_compression=0)
    sheet = book.add_sheet('GRID', cell_overwrite_ok=True)
    
    sheet.write(0, 0, '起始经度')
    sheet.write(0, 1, '起始纬度')
    sheet.write(0, 2, '终止经度')
    sheet.write(0, 3, '终止纬度')
    
    #first grid -- upper-left
    sheet.write(1, 0, minlng)
    sheet.write(1, 1, maxlat)
    sheet.write(1, 2, centerlng)
    sheet.write(1, 3, centerlat)
    
    #second grid -- upper-right
    sheet.write(2, 0, centerlng)
    sheet.write(2, 1, maxlat)
    sheet.write(2, 2, maxlng)
    sheet.write(2, 3, centerlat)
    
    #third grid -- lower-left
    sheet.write(3, 0, minlng)
    sheet.write(3, 1, centerlat)
    sheet.write(3, 2, centerlng)
    sheet.write(3, 3, minlat)
    
    #fourth grid -- lower-right
    sheet.write(4, 0, centerlng)
    sheet.write(4, 1, centerlat)
    sheet.write(4, 2, maxlng)
    sheet.write(4, 3, minlat)
    
    # 最后，将以上操作保存到指定的Excel文件中
    book.save(r'' + str(time) + "次_GRID_"+ adcode + '_' + str(num) +'.xls')

In [8]:
def read_excel(adcode):
    data = pd.read_excel('GRID_' + adcode + '.xls')
    return data

In [9]:
def more_read_excel(adcode, num, time):
    data = pd.read_excel(str(time) + "次_GRID_"+ adcode + '_' + str(num) +'.xls')
    return data

In [10]:
def recursion(time, data):
    for i in range(4):
        record = data.iloc[i]
        minlng = record.iloc[0]
        maxlat = record.iloc[1]
        maxlng = record.iloc[2]
        minlat = record.iloc[3]
        for clas in classes:
            classes_all_pois = []
            counter = 0
            pois_area, counter= getpois(minlng, maxlat, maxlng, minlat, clas, adcode, counter)
            #print('当前城区：' + str(area) + ', 分类：' + str(clas) + ", 总的有" + str(len(pois_area)) + "条数据")
            classes_all_pois.extend(pois_area)
            #print("所有城区的数据汇总，总数为：" + str(len(classes_all_pois)))
            num = i + 1
            if counter < 900:
                
                book = write_to_excel(classes_all_pois, clas, num, adcode, time)
                path = clas + '_' + str(adcode) + '_' + str(num) + '格_' + str(time) + '次' + '.xls'
                auto_save_file(path)
                print('================分类：'  + str(clas) + str(i+1) + '格' + str(time) + "次写入成功")
            else:
                generate_grids(minlng, maxlat, maxlng, minlat, num, adcode, time)
                print(str(time) + '次 格'+ str(num) + ' 数据大于等于900条，已再次分格')
                
                divide_data = more_read_excel(adcode, num, time)
                time = time + 1
                recursion(time, divide_data)
                time = time - 1


In [11]:
import os
import re

In [12]:
def auto_save_file(path):
    directory, file_name = os.path.split(path)
    while os.path.isfile(path):
        pattern = '(\d+)\)\.'
        if re.search(pattern, file_name) is None:
            file_name = file_name.replace('.', '(0).')
        else:
            current_number = int(re.findall(pattern, file_name)[-1])
            new_number = current_number + 1
            file_name = file_name.replace(f'({current_number}).', f'({new_number}).')
        path = os.path.join(directory + os.sep + file_name)
    return path

In [13]:
poi_search_url = "http://restapi.amap.com/v3/place/polygon?"
classes = ['餐饮服务']
amap_web_key = input('Please input your key: ')
str(amap_web_key)
adcode = input('Please input your adcode: ')
adcode = str(adcode)
data = read_excel(adcode)
time = 1 

recursion(time, data)
print('Finish')

Please input your key: c46811a81734366e089404d4b270049e
Please input your adcode: 440106
counter=169
================分类：餐饮服务1格1次写入成功
counter=586
================分类：餐饮服务2格1次写入成功
counter=899
================分类：餐饮服务3格1次写入成功
counter=899
================分类：餐饮服务4格1次写入成功
Finish


In [10]:
poi_search_url = "http://restapi.amap.com/v3/place/polygon?"
classes = ['餐饮服务']
amap_web_key = input('Please input your key: ')
str(amap_web_key)
adcode = input('Please input your adcode: ')
adcode = str(adcode)
data = read_excel(adcode)
time = 1 

for i in range(4):
    record = data.iloc[i]
    minlng = record.iloc[0]
    maxlat = record.iloc[1]
    maxlng = record.iloc[2]
    minlat = record.iloc[3]
    for clas in classes:
        classes_all_pois = []
        counter = 0
        pois_area, counter= getpois(minlng, maxlat, maxlng, minlat, clas, adcode, counter)
        #print('当前城区：' + str(area) + ', 分类：' + str(clas) + ", 总的有" + str(len(pois_area)) + "条数据")
        classes_all_pois.extend(pois_area)
        #print("所有城区的数据汇总，总数为：" + str(len(classes_all_pois)))
        num = i + 1
        if counter < 900:
            write_to_excel(classes_all_pois, clas, num, adcode, time)
            print('================分类：'  + str(clas) + "写入成功")
        else:
            generate_grids(minlng, maxlat, maxlng, minlat, num, adcode, time)
            print(str(time) + '次 File'+ str(num) + ' 数据大于等于900条，已再次分格')
            
            
            
            #-------------------------------------------
            second_data = more_read_excel(adcode, num, time)
            time = time + 1
            for i in range(4):
                record = second_data.iloc[i]
                minlng = record.iloc[0]
                maxlat = record.iloc[1]
                maxlng = record.iloc[2]
                minlat = record.iloc[3]
                for clas in classes:
                    classes_all_pois = []
                    counter = 0
                    pois_area, counter= getpois(minlng, maxlat, maxlng, minlat, clas, adcode, counter)
                    #print('当前城区：' + str(area) + ', 分类：' + str(clas) + ", 总的有" + str(len(pois_area)) + "条数据")
                    classes_all_pois.extend(pois_area)
                    #print("所有城区的数据汇总，总数为：" + str(len(classes_all_pois)))
                    num = i + 1
                    if counter < 900:
                        write_to_excel(classes_all_pois, clas, num, adcode, time)
                        print('================分类：'  + str(clas) + "二次写入成功")
                    else:
                        generate_grids(minlng, maxlat, maxlng, minlat, num, adcode, time)
                        print(str(time) + '次 File'+ str(num) + ' 数据大于等于900条，已再次分格') 
                        
                        
                        
                        #----------------------------------------
                        third_data = more_read_excel(adcode, num, time)
                        time = time + 1
                        for i in range(4):
                            record = second_data.iloc[i]
                            minlng = record.iloc[0]
                            maxlat = record.iloc[1]
                            maxlng = record.iloc[2]
                            minlat = record.iloc[3]
                            for clas in classes:
                                classes_all_pois = []
                                counter = 0
                                pois_area, counter= getpois(minlng, maxlat, maxlng, minlat, clas, adcode, counter)
                                #print('当前城区：' + str(area) + ', 分类：' + str(clas) + ", 总的有" + str(len(pois_area)) + "条数据")
                                classes_all_pois.extend(pois_area)
                                #print("所有城区的数据汇总，总数为：" + str(len(classes_all_pois)))
                                num = i + 1
                                if counter < 900:
                                    write_to_excel(classes_all_pois, clas, num, adcode, time)
                                    print('================分类：'  + str(clas) + "三次写入成功")
                                else:
                                    
                                    generate_grids(minlng, maxlat, maxlng, minlat, num, adcode, time)
                                    print(str(time) + '次 File'+ str(num) + ' 数据大于等于900条，已再次分格 自己想办法解决吧')
    
print('Finish')

Please input your key: 2384eb410a4a717432322a5fa13daf55
Please input your adcode: 440106
counter=155
================分类：餐饮服务写入成功
counter=663
================分类：餐饮服务写入成功
counter=900
1次 File3 数据大于等于900条，已再次分格
counter=882
================分类：餐饮服务二次写入成功
counter=900
2次 File2 数据大于等于900条，已再次分格
counter=882
================分类：餐饮服务三次写入成功
counter=900
3次 File2 数据大于等于900条，已再次分格 自己想办法解决吧
counter=690
================分类：餐饮服务三次写入成功
counter=879
================分类：餐饮服务三次写入成功
counter=690
================分类：餐饮服务二次写入成功
counter=879
================分类：餐饮服务二次写入成功
counter=900
3次 File4 数据大于等于900条，已再次分格
counter=791
================分类：餐饮服务二次写入成功
counter=310
================分类：餐饮服务二次写入成功
counter=897
================分类：餐饮服务二次写入成功
counter=681
================分类：餐饮服务二次写入成功
Finish
